In [1]:
import numpy as np
import pandas as pd
from flask import Flask, render_template, request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import bs4 as bs
import urllib.request
import pickle
import requests
import json

In [2]:
data=pd.read_csv("E:\\Data Science\\Project\\Netflix Recommendation System\\Processed Datasets\\main_data.csv",encoding='latin-1')
data.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,avatar,cch pounder joel david moore wes studi james c...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,pirates of the caribbean: at world's end,johnny depp orlando bloom jack davenport gore ...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,spectre,christoph waltz rory kinnear stephanie sigman ...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,the dark knight rises,tom hardy christian bale joseph gordon-levitt ...
4,doug walker,doug walker,rob walker,unknown,documentary,star wars: episode vii - the force awakens ...,doug walker rob walker unknown doug walker doc...


In [3]:
data['comb']

0       cch pounder joel david moore wes studi james c...
1       johnny depp orlando bloom jack davenport gore ...
2       christoph waltz rory kinnear stephanie sigman ...
3       tom hardy christian bale joseph gordon-levitt ...
4       doug walker rob walker unknown doug walker doc...
                              ...                        
5930     Masi Oka  DJ Khaled  DJ Khaled Nick Bruno, Tr...
5931     Chris Cooper  Louis Garrel  Louis Garrel Gret...
5932     Benedict Cumberbatch  Colin Firth  Colin Firt...
5933     C.J. LeBlanc  O'Shea Jackson Jr.  O'Shea Jack...
5934     Danielle Brooks  Richard Schiff  Richard Schi...
Name: comb, Length: 5935, dtype: object

In [4]:
data.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [5]:
count=CountVectorizer()
count_matrix=count.fit_transform(data['comb'])

In [6]:
cosine_sim=cosine_similarity(count_matrix,count_matrix)
cosine_sim

array([[1.        , 0.24174689, 0.16116459, ..., 0.06900656, 0.        ,
        0.        ],
       [0.24174689, 1.        , 0.18181818, ..., 0.07784989, 0.        ,
        0.        ],
       [0.16116459, 0.18181818, 1.        , ..., 0.23354968, 0.        ,
        0.        ],
       ...,
       [0.06900656, 0.07784989, 0.23354968, ..., 1.        , 0.05923489,
        0.07161149],
       [0.        , 0.        , 0.        , ..., 0.05923489, 1.        ,
        0.06362848],
       [0.        , 0.        , 0.        , ..., 0.07161149, 0.06362848,
        1.        ]])

In [7]:
data.index

RangeIndex(start=0, stop=5935, step=1)

In [8]:
titles=pd.Series(data['movie_title'])
titles

0                                                  avatar
1                pirates of the caribbean: at world's end
2                                                 spectre
3                                   the dark knight rises
4       star wars: episode vii - the force awakens    ...
                              ...                        
5930                                    spies in disguise
5931                                         little women
5932                                                 1917
5933                                           just mercy
5934                                             clemency
Name: movie_title, Length: 5935, dtype: object

In [9]:
def recommendations(title,cosine_sim=cosine_sim):
    recommend_movies=[]
    recommend=[]
    title=title.lower()
    idx=titles[titles==title].index[0] # getting the index of movie that matches the title
    score_series=pd.Series(cosine_sim[idx]).sort_values(ascending=False) # creating a series with the similarity scores in descending order
    top_10_indexes=list(score_series.iloc[1:11].index) # getting the indexes of the 10 most similar movies
    for i in top_10_indexes:
      recommend_movies.append(list(data['movie_title'])[i])

    return recommend_movies

In [14]:
recommendations('the amazing spider-man')

['the amazing spider-man 2',
 'thor',
 'wonder woman',
 'the imaginarium of doctor parnassus',
 'thor: the dark world',
 'harry potter and the chamber of secrets',
 'snow white and the huntsman',
 'epic',
 'shrek',
 "the huntsman: winter's war"]